## Spark Connector using Apache Arrow Format
The Snowflake Connector for Spark (“Spark Connector”) now uses the Apache Arrow 
columnar result format to dramatically improve query read performance.

With this 2.6.0 release, the Snowflake Spark Connector executes the query directly 
via JDBC and (de)serializes the data using Arrow, Snowflake’s new client result 
format. This saves time in data reads and also enables the use of cached query results.

In [ ]:
// Enter the values of account, user, password, warehouse and role to match your environment
val ACCOUNT = "<snowflake_account>.snowflakecomputing.com"
val USER =  "<snowflake_user_name>"
val PASSWORD =  "<snowflake_password>"
val WAREHOUSE = "<snowflake_warehouse>"
val DATABASE = "SNOWFLAKE_SAMPLE_DATA"

In [ ]:
// Snowflake Spark Connector options
// The default value of "use_cached_result" is "false".
// It is "true" means to disable this feature.
import net.snowflake.spark.snowflake._

val sfOptions: Map[String, String] = Map(
"use_copy_unload" -> "false",  
"use_cached_result" -> "false",
"partition_size_in_mb" -> "60",   
"sfSSL" -> "on",
"sfUser" -> USER,
"sfPassword" -> PASSWORD,
"sfDatabase" -> DATABASE,
"sfURL" -> ACCOUNT,
"sfWarehouse" -> WAREHOUSE
)

In [ ]:
import org.apache.spark.sql._

val sqlContext = new SQLContext(sc) 
// Test table is TPCH LINEITEM which has 600M rows.
// Its compressed size in Snowflake is 16.3GB.
val sourceTableName = "LINEITEM"
val sourceSchema = "TPCH_SF100"

val df = sqlContext.read.format("net.snowflake.spark.snowflake")
.options(sfOptions)
.option("dbtable", sourceTableName)
.option("sfSchema", sourceSchema)
.load()


In [ ]:
// Clear cache before any test
sqlContext.clearCache()
// Execute cache() and show(1) to read all data into cache
// and show one row. The time to show one row can be ignored.
// The DataFrame execution time is regarded as the reading time.
val startTime = System.currentTimeMillis()
df.cache().show(1)
val endTime = System.currentTimeMillis()

val result_msg = s"read time: ${(endTime - startTime).toDouble/1000.0} s"
println(result_msg)